In [1]:

# Import necessary libraries
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression 
from sklearn.metrics import accuracy_score,log_loss, median_absolute_error
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import VotingClassifier, BaggingClassifier,BaggingRegressor
from sklearn.neighbors import *
from sklearn.svm import SVC
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os

warnings.simplefilter('ignore')

os.chdir('D:\kaggla_comp\hardness')

train = pd.read_csv('train.csv',index_col=0)
train

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8


In [2]:
x= train.drop('Hardness',axis=1)
y=train['Hardness']

In [3]:
# Create a K-Fold cross-validation object with 5 splits, shuffling the data, and a random state of 24
kfold = KFold(n_splits=5, shuffle=True, random_state=24)

# Create a Decision Tree Regressor
dtr = DecisionTreeRegressor()

# Create a Ridge Regression model
ridge = Ridge()

# Create a Linear Regression model
lr = LinearRegression()

# Create a Bagging Regressor using Linear Regression as the base estimator
# Set the number of estimators to 15 and enable out-of-bag (OOB) score calculation
bagg = BaggingRegressor(estimator=lr, n_estimators=15, random_state=24, oob_score=True)

# Define the parameter grid for the Bagging Regressor
params = {
    'estimator': [lr, ridge, dtr],
    'n_estimators': [10, 15]
}

# Create a GridSearchCV object to perform cross-validation and hyperparameter tuning
# Use the Bagging Regressor as the estimator, the parameter grid defined above, the K-Fold cross-validation,
# set the verbosity to 3, and use the negative median absolute error as the scoring metric
gcv = GridSearchCV(bagg, param_grid=params, cv=kfold, verbose=3, scoring='neg_median_absolute_error')

# Fit the GridSearchCV object to the entire dataset (x and y)
gcv.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv.best_params_)

# Print the best score (negative median absolute error) found by the GridSearchCV
print("Best Score:", gcv.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END estimator=LinearRegression(), n_estimators=10;, score=-0.978 total time=   0.0s
[CV 2/5] END estimator=LinearRegression(), n_estimators=10;, score=-0.970 total time=   0.0s
[CV 3/5] END estimator=LinearRegression(), n_estimators=10;, score=-0.955 total time=   0.0s
[CV 4/5] END estimator=LinearRegression(), n_estimators=10;, score=-0.978 total time=   0.0s
[CV 5/5] END estimator=LinearRegression(), n_estimators=10;, score=-0.928 total time=   0.0s
[CV 1/5] END estimator=LinearRegression(), n_estimators=15;, score=-0.976 total time=   0.0s
[CV 2/5] END estimator=LinearRegression(), n_estimators=15;, score=-0.968 total time=   0.0s
[CV 3/5] END estimator=LinearRegression(), n_estimators=15;, score=-0.960 total time=   0.0s
[CV 4/5] END estimator=LinearRegression(), n_estimators=15;, score=-0.980 total time=   0.0s
[CV 5/5] END estimator=LinearRegression(), n_estimators=15;, score=-0.933 total time=   0.0s
[CV 1/5] E

In [4]:
# Create a Bagging Regressor using Decision Tree Regressor as the base estimator
# Set the random state to 24
bagg = BaggingRegressor(estimator=dtr, random_state=24)

# Print the parameters of the Bagging Regressor
print(bagg.get_params())

{'base_estimator': 'deprecated', 'bootstrap': True, 'bootstrap_features': False, 'estimator__ccp_alpha': 0.0, 'estimator__criterion': 'squared_error', 'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__max_leaf_nodes': None, 'estimator__min_impurity_decrease': 0.0, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__min_weight_fraction_leaf': 0.0, 'estimator__random_state': None, 'estimator__splitter': 'best', 'estimator': DecisionTreeRegressor(), 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 10, 'n_jobs': None, 'oob_score': False, 'random_state': 24, 'verbose': 0, 'warm_start': False}


In [6]:
# Define the parameter grid for the Bagging Regressor
params = {
    'estimator__max_depth': [None, 3, 5],
    'estimator__min_samples_leaf': [1, 5, 10],
    'estimator__min_samples_split': [2, 5, 10],
    'n_estimators': [15, 30, 50]
}

# Create a GridSearchCV object to perform cross-validation and hyperparameter tuning
# Use the Bagging Regressor as the estimator, the parameter grid defined above, the K-Fold cross-validation,
# set the verbosity to 3
gcv_s = GridSearchCV(bagg, param_grid=params, cv=kfold, verbose=3)

# Fit the GridSearchCV object to the entire dataset (x and y)
gcv_s.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv_s.best_params_)

# Print the best score found by the GridSearchCV
print("Best Score:", gcv_s.best_score_)

In [25]:
best_model = gcv.best_estimator_


test_data = pd.read_csv("test.csv",index_col=0)

pred_berry = best_model.predict(test_data)


submit = pd.read_csv("sample_submission.csv")


submit['Hardness'] = pred_berry

# Save the submission file with the predicted hardness values
submit.to_csv("D:\kaggla_comp\hardnessbootstrap_hard_25.csv", index=False)